In [ ]:
using PerlaTonettiWaugh, LinearAlgebra, Plots, BenchmarkTools, Interpolations, Roots, Test, Base.Iterators

Set up parameters and find the corresponding stationary solution:

In [ ]:
z_min = 0.0 
z_max = 5.0
M = 1000
z_grid = range(z_min, stop = z_max, length = M) # Since we only care about the grid. 

# Define common objects. 
d_0 = 5
d_T = 2.3701
params = (ρ = 0.02, σ = 4.2508, N = 10, θ = 5.1269, γ = 1.00, κ = 0.013, ζ = 1, η = 0, Theta = 1, χ = 1/(2.1868), υ = 0.0593, μ = 0, δ = 0.053) # Baselines per Jesse. 

# solve for stationary solution at t = 0
params_0 = merge(params, (d = d_0,)) # parameters to be used at t = 0
params_T = merge(params, (d = d_T,)) # parameters to be used at t = T

stationary_sol_0 = stationary_numerical(params_0, z_grid) # solution at t = 0
stationary_sol = stationary_numerical(params_T, z_grid) # solution at t = T

Ω_0 = stationary_sol_0.Ω
Ω_T = stationary_sol.Ω
settings = (z = z_grid, tstops = nothing, Δ_E = 1e-06)

println("g = $(stationary_sol.g), z_hat = $(stationary_sol.z_hat), Ω = $(stationary_sol.Ω)")

Define the objective function:

In [ ]:
function solve_with_candidate(candidate)
    candidate = [candidate...] # if candidate is a tuple, convert it to an array
    T = candidate[end]
    
    
    # construct Ω and E
    Ω = PolynomialΩ(candidate, Ω_0, Ω_T, params.δ)
    E = Ω.E

    # check if E is increasing in the intervals (if not, it means it is malfunctioning i.e. return Inf)
    if (!is_positive_in_interval(Ω.E_derivative, 0.0, T)) throw("E is not increasing in [0,T].") end

    # solve the dynamics and get the resulting entry_residual vector; if solution is not valid, return Inf
    return solve_dynamics(params_T, stationary_sol, settings, T, Ω, E)
end

function evaluate_candidate(candidate)
    candidate = [candidate...] # if candidate is a tuple, convert it to an array
    T = candidate[end]
    
    # construct Ω and E
    Ω = PolynomialΩ(candidate, Ω_0, Ω_T, params.δ)
    E = Ω.E

    # check if E is increasing in the intervals (if not, it means it is malfunctioning i.e. return Inf)
    if (!is_positive_in_interval(Ω.E_derivative, 0.0, T)) return Inf end

    # solve the dynamics and get the resulting entry_residual vector; if solution is not valid, return Inf
    solved = try solve_dynamics(params_T, stationary_sol, settings, T, Ω, E).results catch; return Inf end
    
    t = solved.t
    entry_residual = solved.entry_residual

    # interpolate on returned entry_residual
    entry_residual_interpolated = LinearInterpolation(t, entry_residual)

    # evaluate entry_residual on entry_residual_nodes, return the norm
    entry_residuals_nodes = range(0, stop = T, length = entry_residuals_nodes_count + 2)
    return (norm(entry_residual_interpolated.(entry_residuals_nodes[2:(end-1)])))
end

Define candidates:

In [ ]:
c1_candidates = range(-2e-6, stop = -1e-6, length = 5)
c2_candidates = range(-3e-6, stop = 0.0, length = 5)
T_candidates = range(20.0, stop = 30.0, length = 5)
entry_residuals_nodes_count = 10

# generate an array of candidates where the last element of each candidate is a candidate for T
candidates = product(c1_candidates, c2_candidates, T_candidates)

Show solution per each candidate:

In [ ]:
performances = evaluate_candidate.(candidates)

Find the corresponding solution

In [ ]:
solution = (x->x).(candidates)[argmin(performances)]

In [ ]:
solved = solve_with_candidate(solution)

v_t0 = solved.sol.u[1][1:M]
v0 = solved.results[:v_0]
# save v0 and v_hat_t0
v_hat_t0 = map(z -> exp((params.σ-1)*z), z_grid) .* v_t0;

solved = solved.results

## Plots for Ω and `entry_residuals`

In [ ]:
plot_Ω = plot(solved.t, solved.Ω, label = "Omega", lw = 3)
plot_residual = plot(solved.t, solved.entry_residual, label = "entry_residual", lw = 3)
plot(plot_Ω, plot_residual, layout = (2,1))

In [ ]:
norm(solved.entry_residual)

## Primary Plots

In [ ]:
plot1 = plot(solved.t, solved.g, label = "g", lw = 3)
plot2 = plot(solved.t, solved.z_hat, label = "z_hat", lw = 3)
plot3 = plot(solved.t, solved.S, label = "S", lw = 3)
plot4 = plot(solved.t, solved.entry_residual, label = "entry_residual", lw = 3)
plot(plot1, plot2, plot3, plot4, layout=(2,2))

## Static Equations

In [ ]:
plot1 = plot(solved.t, solved.L_tilde, label = "L_tilde", lw = 3)
plot2 = plot(solved.t, solved.z_bar, label = "z_bar", lw = 3)
plot3 = plot(solved.t, solved.π_min, label = "pi_min", lw = 3)
plot4 = plot(solved.t, solved.λ_ii, label = "lambda_ii", lw = 3)
plot5 = plot(solved.t, solved.c, label = "c", lw = 3)
plot6 = plot(solved.t, solved.E, label = "E", lw = 3)
plot7 = plot(solved.t, solved.log_M, label = "log_M", lw = 3)
plot8 = plot(solved.t, solved.U, label = "U", lw = 3)
plot9 = plot(z_grid, v_hat_t0, label = "v_hat at t = 0", lw = 3)
plot(plot1, plot2, plot3, plot4, plot5, plot6, plot7, plot8, layout=(4,2))

In [ ]:
# Can examine the returned data with the Voyager and/or Vegalite
using DataVoyager, VegaLite
solved |> Voyager()
#solved |> @vlplot(:line, x = :t, y = :g, width=400, height=400)